#### Import necessary libraries 

In [4]:
!pip install deepchem==2.6.1
!pip install rdkit-pypi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
# importing utility modules
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
 
# importing machine learning models for prediction
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier 
# importing voting classifier
from sklearn.ensemble import VotingClassifier
import deepchem as dc
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from deepchem import metrics

import numpy as np
from sklearn.metrics import matthews_corrcoef
from deepchem.splits import RandomSplitter
from scipy.stats import ttest_ind

#### Read in the preprocessed BBB dataset from Adenot paper

In [6]:
bbb_df = pd.read_csv('adenot_processed.csv')

In [7]:
bbb_df.head()

,Drug,SMILES,permeable,0,1,2,3,4,5,6,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,ACEBUTOLOL,CCCC(=O)Nc1ccc(c(c1)C(C)=O)OCC(O)CNC(C)C,0,0,1,0,0,1,0,0,...,1,0,0,1,0,0,0,0,0,0
1,DACTINOMYCIN,CC(C)[C@H]1NC(=O)[C@@H](NC(=O)c2ccc(c3c2N=C2C(...,0,0,1,0,0,1,1,0,...,0,0,0,0,0,1,0,0,0,0
2,ALDOSTERONE,C[C@@]12CCC(=O)C=C2CC[C@H]2C3CC[C@H](C(=O)CO)C...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,AMILORIDE,N\C(=N)\NC(=O)c1nc(c(nc1N)N)Cl,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,AMIODARONE,CCCCc1oc2ccccc2c1C(=O)c1cc(c(c(c1)[I])OCCN(CC)...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Data preprocessing

In [8]:
X = bbb_df.iloc[:,3:].copy()
y = bbb_df.iloc[:,2].copy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

## Optimized Models

#### Optimized Random Forest

In [9]:
# Create a Random Forest Classifier
rf_best = RandomForestClassifier(random_state=0, n_estimators = 100, criterion='gini', max_depth=20)

# Train the model using the training sets
rf_best.fit(X_train,y_train)

RandomForestClassifier(max_depth=20, random_state=0)

#### Optimized SVM

In [10]:
# Create a Support Vector Machine Classifier
SVM_best = SVC(C=0.1, gamma=1, kernel='linear', probability=True).fit(X_train, y_train)

# Train the model using the training sets
SVM_best.fit(X_train, y_train)

SVC(C=0.1, gamma=1, kernel='linear', probability=True)

#### Optimized XGBoost

In [11]:
xg_best = XGBClassifier(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=4,
 min_child_weight=3,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.85,
 reg_alpha=1e-05,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,)

In [12]:
xg_best.fit(X_train, y_train)

XGBClassifier(colsample_bytree=0.85, max_depth=4, min_child_weight=3,
              n_estimators=1000, nthread=4, reg_alpha=1e-05, subsample=0.8)

### Ensemble

In [13]:
# Making the final model using voting classifier
final_model = VotingClassifier(
    estimators=[('svm', SVM_best), ('xgb', xg_best), ('rf', rf_best)], voting='soft')
 
# training all the model on the train dataset
final_model.fit(X_train, y_train)


VotingClassifier(estimators=[('svm',
                              SVC(C=0.1, gamma=1, kernel='linear',
                                  probability=True)),
                             ('xgb',
                              XGBClassifier(colsample_bytree=0.85, max_depth=4,
                                            min_child_weight=3,
                                            n_estimators=1000, nthread=4,
                                            reg_alpha=1e-05, subsample=0.8)),
                             ('rf',
                              RandomForestClassifier(max_depth=20,
                                                     random_state=0))],
                 voting='soft')

## Data Preprocessing

In [14]:
Xs = bbb_df.iloc[:,3:].copy()
Ys = bbb_df.iloc[:,2].copy()
dataset = dc.data.DiskDataset.from_numpy(X=Xs,y=Ys,ids=bbb_df['SMILES'].tolist())
scaffoldsplitter = dc.splits.ScaffoldSplitter()

## K-Fold and MCC Caluclations

In [15]:
def K_fold_MCC(dataset, h, split_name="Random Split", splitter=RandomSplitter()):
    
    split_data = splitter.k_fold_split(dataset, k=4)
    
    MCCs = []
    y_true = []
    h_predictions = []
    
    for data in split_data:
        h.fit(data[0].X, data[0].y)
        y_pred = h.predict(data[1].X)
        y_true.extend(list(data[1].y))
        h_predictions.extend(list(y_pred))
        mcc = matthews_corrcoef(data[1].y, y_pred)
        MCCs.append(mcc)
    
    print(split_name + " MCC Values:")
    
    for mcc in MCCs:
        print(mcc)
    print("Mean: " + str(np.mean(MCCs)))

    print("MCC value across full test data: " + str(matthews_corrcoef(y_true, h_predictions)))
    
    return MCCs

#### Comparing MCC Values

In [16]:
model_MCC_dict = {}

#### MCC values for RF Model

In [17]:
scaffold_split_mcc = K_fold_MCC(dataset, rf_best, 'Scaffold Split', scaffoldsplitter)
random_split_mcc = K_fold_MCC(dataset=dataset, h=rf_best)
model_MCC_dict["Random Forest"] = (scaffold_split_mcc, random_split_mcc)

<ipython-input-15-37d8496c44a6>:10: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  h.fit(data[0].X, data[0].y)


Scaffold Split MCC Values:
0.7846914939903075
0.807007127726689
0.8145075485434284
0.6801964731777397
Mean: 0.7716006608595412
MCC value across full test data: 0.7700602690681662


<ipython-input-15-37d8496c44a6>:10: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  h.fit(data[0].X, data[0].y)


Random Split MCC Values:
0.8039164733622468
0.8591163475679805
0.8078691652767769
0.8450526965562208
Mean: 0.8289886706908063
MCC value across full test data: 0.8296004851692456


#### MCC values for SVM Model

In [18]:
scaffold_split_mcc = K_fold_MCC(dataset, SVM_best, 'Scaffold Split', scaffoldsplitter)
random_split_mcc = K_fold_MCC(dataset=dataset, h=SVM_best)
model_MCC_dict["SVM"] = (scaffold_split_mcc, random_split_mcc)

/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Scaffold Split MCC Values:
0.6887294242922333
0.8368969503903855
0.3739754258990208
0.7728042598325765
Mean: 0.668101515103554
MCC value across full test data: 0.7598248879287592


/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Random Split MCC Values:
0.8888284216190426
0.8414307648225654
0.8647836831736363
0.844498433016823
Mean: 0.8598853256580169
MCC value across full test data: 0.8596961674100803


#### MCC values for XG-Boost Model

In [19]:
scaffold_split_mcc = K_fold_MCC(dataset, xg_best, 'Scaffold Split', scaffoldsplitter)
random_split_mcc = K_fold_MCC(dataset=dataset, h=xg_best)
model_MCC_dict["XG-Boost"] = (scaffold_split_mcc, random_split_mcc)

/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Scaffold Split MCC Values:
0.6515187123488106
0.811764902392185
0.35340999030727216
0.7561700932611006
Mean: 0.6432159245773421
MCC value across full test data: 0.7337132712263612


/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Random Split MCC Values:
0.8779480181349626
0.8322639421600209
0.8481109305196614
0.8545892311033624
Mean: 0.8532280304795018
MCC value across full test data: 0.8519899660914146


#### MCC values for Ensemble Model

In [20]:
scaffold_split_mcc = K_fold_MCC(dataset, final_model, 'Scaffold Split', scaffoldsplitter)
random_split_mcc = K_fold_MCC(dataset=dataset, h=final_model)
model_MCC_dict["Ensemble"] = (scaffold_split_mcc, random_split_mcc)

/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Scaffold Split MCC Values:
0.7586646256039993
0.8361088119466458
0.5094404799339134
0.7657810101349732
Mean: 0.7174987319048829
MCC value across full test data: 0.7920621838460163


/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Random Split MCC Values:
0.8150578479844451
0.8632739571224687
0.9344227257377159
0.8554452324884078
Mean: 0.8670499408332594
MCC value across full test data: 0.8675840977983985


#### Statistical Comparison

In [21]:
splits = ["Scaffold Split", "Random Split"]

for i in range(2):
    print(splits[i] + " P Values")
    for model in model_MCC_dict:
        for model_2 in model_MCC_dict:
            if model != model_2:
                print(model + " vs. " + model_2 + ": " + str(ttest_ind(model_MCC_dict[model][i], model_MCC_dict[model_2][i]).pvalue))
        if model != model_2:
            print()
    print("____________________")
    print()

Scaffold Split P Values
Random Forest vs. SVM: 0.37176739411050375
Random Forest vs. XG-Boost: 0.2745614587376995
Random Forest vs. Ensemble: 0.513852352990134

SVM vs. Random Forest: 0.37176739411050375
SVM vs. XG-Boost: 0.8691987168817623
SVM vs. Ensemble: 0.7065791667650955

XG-Boost vs. Random Forest: 0.2745614587376995
XG-Boost vs. SVM: 0.8691987168817623
XG-Boost vs. Ensemble: 0.5731543949108158

Ensemble vs. Random Forest: 0.513852352990134
Ensemble vs. SVM: 0.7065791667650955
Ensemble vs. XG-Boost: 0.5731543949108158
____________________

Random Split P Values
Random Forest vs. SVM: 0.12810111536197724
Random Forest vs. XG-Boost: 0.19524517098560756
Random Forest vs. Ensemble: 0.22770233065956388

SVM vs. Random Forest: 0.12810111536197724
SVM vs. XG-Boost: 0.6619765693732391
SVM vs. Ensemble: 0.8005268100343743

XG-Boost vs. Random Forest: 0.19524517098560756
XG-Boost vs. SVM: 0.6619765693732391
XG-Boost vs. Ensemble: 0.6215121397637677

Ensemble vs. Random Forest: 0.227702330